
# Lab 3: Using Mirror's Features
## Using Mirror feature on volumes for disaster recovery or development use.

### With Data Fabric Mirroring you can:

- Mirror data from one cluster to another for disaster recovery (not showcased here today)
- Mirror data localy on one cluster for recovery, development, testing or data validation needs.
- Schedules mirrors or run them on demand.
- Promote mirrors out of read-only mode to read-write

<p align="center">
  <img src="Pictures/edf-mirror-overview.png" width="600" height="400" alt="Data Fabric Snapshots">                                                    
  
</p>

In [1]:
%login 16.31.86.198

echo "We are now connected to the appliance"

#Variables used by scripts
#password="qcpx0k4u" # your Data Fabric login credentials - password - 
#apiserver_host="16.31.85.156" # API Server for the Data Fabric Cluster
#ad_group="data-fabric-users" # LDAP Domain Group for Data Fabric users
#vol_name="vol1" # suffix for student volume name
#snapvol_name="snapvol1" # suffix for student snap volume name
#cluster_name="mapr-cluster2.etc.fr.comm.hpecorp.net" # Data Fabric Cluster Name
#src_vol_name="src-mirrvol1" # suffix for student source mirror volume name
#dst_vol_name="dst-mirrvol1"  # suffix for student destination mirror volume name
#secvol_name="sec-vol1"  # suffix for student volume name used for security comparison / user access 
#sec_policy="student1222-policy" #security policy to allow r/w access to volumes

echo "your operation context is:" student1222 "with Password:" qcpx0k4u

[ssh] Login to 16.31.86.198...
[ssh] host=16.31.86.198 hostname=16.31.86.198 other_conf={'user': None, 'port': None, 'keyfile': None, 'load_system_ssh_config': False, 'missing_host_policy': <paramiko.client.WarningPolicy object at 0x7fdbf0158520>}
[ssh] Successfully logged in.
[ssh] host = 16.31.86.198, cwd = /student/student1222
We are now connected to the appliance
your operation context is: student1222 with Password: qcpx0k4u


## 1. Creating a source volume that we will use to mirror from.

#### Background:

You have been given a user that is allowed to create and modify volumes on the cluster.  Data Fabric volumes can be mounted so they are accessable from NFS, HDFS or POSIX commands (like the bash kernel shell in these labs).  For security reasons, you can only mount volumes in the /user directory.

Fist let's create a volume to create mirror's on:

In [2]:
source ~/.bashrc

curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/create?name='$username-$src_vol_name'&path=/user/'$username-$src_vol_name'' --user $username:$password

[ssh] host = 16.31.86.198, cwd = /student/student1222
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   182  100   182    0     0     73      0  0:00:02  0:00:02 --:--:--    73
{"timestamp":1624511577959,"timeofday":"2021-06-24 07:12:57.959 GMT+0200 AM","status":"OK","total":0,"data":[],"messages":["Successfully created volume: 'student1222-src-mirrvol1'"]}

## 2. Create a mirror volume from your source volume.

Now let's create a local mirror volume from our source volume we made in the previous step:

In [3]:
source ~/.bashrc

curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/create?name='$username-$dst_vol_name'&path=/user/'$username-$dst_vol_name'&type=mirror&source='$username-$src_vol_name'@'$cluster_name'' --user $username:$password

[ssh] host = 16.31.86.198, cwd = /student/student1222
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   182  100   182    0     0    409      0 --:--:-- --:--:-- --:--:--   410
{"timestamp":1624511578491,"timeofday":"2021-06-24 07:12:58.491 GMT+0200 AM","status":"OK","total":0,"data":[],"messages":["Successfully created volume: 'student1222-dst-mirrvol1'"]}

## 3. Create data within your source volume

Now let's create some data in our source volume so we can verify that the mirror operation worked:

In [4]:
source ~/.bashrc
echo "## Create a file in your source volume that we will mirror/replicate in a later step.##"
echo "$username-$vol_name source volume update $(date)" >> /mapr/$cluster_name/user/$username-$src_vol_name/$username-source-volume-file1
echo "## Verify the file you created in in your directory.##"
ls /mapr/$cluster_name/user/$username-$src_vol_name
echo "## Verify the file has the contents you added.##"
cat /mapr/$cluster_name/user/$username-$src_vol_name/$username'-'source-volume-file1

[ssh] host = 16.31.86.198, cwd = /student/student1222
## Create a file in your source volume that we will mirror/replicate in a later step.##
## Verify the file you created in in your directory.##
student1222-source-volume-file1
## Verify the file has the contents you added.##
student1222-vol1 source volume update Thu Jun 24 07:12:23 CEST 2021


## 4. Start the mirroring process

By default a target or destination mirror volume is mounted as read-only (ro).  Also by default the mirror is not started until you manually start it or apply a schedule to the mirror.  In this example we will be starting it manually.

**Use `CTRL+Enter` on this cell to run it again in case mirroring not done yet.**

In [5]:
source ~/.bashrc

curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/mirror/start?name='$username-$dst_vol_name'' --user $username:$password

[ssh] host = 16.31.86.198, cwd = /student/student1222
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   192  100   192    0     0    692      0 --:--:-- --:--:-- --:--:--   693
{"timestamp":1624511578968,"timeofday":"2021-06-24 07:12:58.968 GMT+0200 AM","status":"OK","total":1,"data":[{"messages":"Started mirror operation for volume(s) 'student1222-dst-mirrvol1' "}]}

This lab is a short term setup of managing data so we will not cover setting up schedules and applying them to snapshot's or to mirrors.  For more details on creating schedules see: [Creating Schedules on Ezmeral Data Fabric](https://docs.datafabric.hpe.com/62/ReferenceGuide/schedule.html)

## 5. Verify your source data is now within your destination volume

Now let's verify the data we wrote in our previous steps in our source volume is now in the destination volume and that our mirror operation we started worked:

In [6]:
source ~/.bashrc
echo "## List out the directory contents on our destination volume.##"
ls /mapr/$cluster_name/user/$username-$dst_vol_name

[ssh] host = 16.31.86.198, cwd = /student/student1222
## List out the directory contents on our destination volume.##


Now let's write some data to your new destination volume that was mirrored from the source volume:


In [7]:
source ~/.bashrc
echo "##Let's try and write some data to our destination volume...##"
echo "$username-$vol_name destination volume update $(date)" >> /mapr/$cluster_name/user/$username-$dst_vol_name/$username-source-volume-file1
echo "##List out the destination volume directory.##"
ls /mapr/$cluster_name/user/$username-$dst_vol_name
echo "##Verify the file we wrote is on the destination volume directory.##"
cat /mapr/$cluster_name/user/$username-$dst_vol_name/$username-source-volume-file1

[ssh] host = 16.31.86.198, cwd = /student/student1222
##Let's try and write some data to our destination volume...##
/usr/bin/bash: line 3: /mapr/mapr-cluster2.etc.fr.comm.hpecorp.net/user/student1222-dst-mirrvol1/student1222-source-volume-file1: Read-only file system
##List out the destination volume directory.##
##Verify the file we wrote is on the destination volume directory.##
cat: /mapr/mapr-cluster2.etc.fr.comm.hpecorp.net/user/student1222-dst-mirrvol1/student1222-source-volume-file1: No such file or directory


abnormal exit code: 1

**Notice the update to the file did not have the “destination volume update” appended to the file.** This error is to be expected as by default destination volumes that are currently being mirrored are mounted in read-only mode (ro). Let's break the mirror and mount the volume in read-write (rw) mode:

## 6. Promoting destination volumes to read-write

First let's ensure the mirror process has stoped:

In [8]:
source ~/.bashrc

curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/mirror/stop?name='$username-$dst_vol_name'' --user $username:$password

[ssh] host = 16.31.86.198, cwd = /student/student1222
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   182  100   182    0     0    558      0 --:--:-- --:--:-- --:--:--   558
{"timestamp":1624511580203,"timeofday":"2021-06-24 07:13:00.203 GMT+0200 AM","status":"OK","total":1,"data":[{"messages":"Stopped mirror operation for 'student1222-dst-mirrvol1' "}]}

Now we can change the volume mount to read-write:

In [9]:
source ~/.bashrc

curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/modify?name='$username-$dst_vol_name'&type=rw' --user $username:$password

[ssh] host = 16.31.86.198, cwd = /student/student1222
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   111  100   111    0     0    379      0 --:--:-- --:--:-- --:--:--   380
{"timestamp":1624511580733,"timeofday":"2021-06-24 07:13:00.733 GMT+0200 AM","status":"OK","total":0,"data":[]}

Finally we can verify that the new destination volume has been mounted read-write (rw) by writing new data to it:

In [10]:
source ~/.bashrc
echo "##Now let's try and write some data to our destination volume...##"
echo "$username-$vol_name destination volume update $(date)" >> /mapr/$cluster_name/user/$username-$dst_vol_name/$username-source-volume-file1
echo "##List out the destination volume directory.##"
ls /mapr/$cluster_name/user/$username-$dst_vol_name
echo "##Verify the file we wrote is on the destination volume directory.##"
cat /mapr/$cluster_name/user/$username-$dst_vol_name/$username-source-volume-file1

[ssh] host = 16.31.86.198, cwd = /student/student1222
##Now let's try and write some data to our destination volume...##
/usr/bin/bash: line 3: /mapr/mapr-cluster2.etc.fr.comm.hpecorp.net/user/student1222-dst-mirrvol1/student1222-source-volume-file1: Read-only file system
##List out the destination volume directory.##
##Verify the file we wrote is on the destination volume directory.##
cat: /mapr/mapr-cluster2.etc.fr.comm.hpecorp.net/user/student1222-dst-mirrvol1/student1222-source-volume-file1: No such file or directory


abnormal exit code: 1

You will notice that our file now has the “destination volume update” appended to the file.  This confirms that the destination mirror volume is mounted and working in read-write (rw) mode.

For additional information on using HPE Ezmeral Data Fabric Mirror's Features see: [Mirror Volumes](https://docs.datafabric.hpe.com/62/AdministratorGuide/MirrorVolumes-Mirroring.html)

## 7. Removing mirrors
### Let's cleanup now

 1. Remove the Mirror  volumes
 2. Remove the Source Volume
 
 **Note the `&force=1 here`.  Because Mirror volumes actually use snapshots. It will error out saying there are snapshots you need to remove first. This will just force a removal so we don’t have to remove the snapshots**

In [11]:
source ~/.bashrc

echo "Deleting dstmirror snapshot"
snapiddstmirror=$(curl -k --user $username:$password -X POST 'https://'$apiserver_host':8443/rest/volume/snapshot/list'| jq -r | grep -B 12 -A 2 ''$username-$dst_vol_name'' | grep snapshotid | cut -c22-30)
echo $snapiddstmirror
curl -k --user $username:$password -X POST 'https://'$apiserver_host':8443/rest/volume/snapshot/remove?snapshots='$snapiddstmirror

echo "Deleting srcmirror snapshot"
snapidsrcmirror=$(curl -k --user $username:$password -X POST 'https://'$apiserver_host':8443/rest/volume/snapshot/list'| jq -r | grep -B 12 -A 2 ''$username-$src_vol_name'' | grep snapshotid | cut -c22-30)
echo $snapidsrcmirror
curl -k --user $username:$password -X POST 'https://'$apiserver_host':8443/rest/volume/snapshot/remove?snapshots='$snapidsrcmirror

echo "Removing the Mirror Volume"
curl -k --user $username:$password -X POST 'https://'$apiserver_host':8443/rest/volume/remove?name='$username-$dst_vol_name -force

echo "Removing the Source Volume"
curl -k --user $username:$password -X POST 'https://'$apiserver_host':8443/rest/volume/remove?name='$username-$src_vol_name -force


[ssh] host = 16.31.86.198, cwd = /student/student1222
Deleting dstmirror snapshot
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   696  100   696    0     0   2343      0 --:--:-- --:--:-- --:--:--  2351

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   151  100   151    0     0    576      0 --:--:-- --:--:-- --:--:--   578
{"timestamp":1624511583700,"timeofday":"2021-06-24 07:13:03.700 GMT+0200 AM","status":"ERROR","errors":[{"id":22,"desc":"Error parsing snapshot IDs"}]}Deleting srcmirror snapshot
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   696  100   696    0     0   2282      0 --:--:-- --:--:-- --:--:--  2281
256000074
  % Total    % Re

In [12]:
%logout 

[ssh] Closing existing connection.
[ssh] Successfully logged out.


## Mirror's Summary

In this example we:

1. Created a new source volume
2. Wrote data to it
3. Setup a new destination mirror volume
4. Started the mirror process on the destination volume.
5. And then promoted that mirror to be it's own volume and wrote new data to it.

# Using Access Security Settings and Policies  on your data
You can now move to the conclusion of this workshop !

* [Lab 4 - Using Access Security Settings and Policies](4-WKSHP-App-Security-Policies.ipynb)